In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_5%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,NaN,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,NaN,3.18,7.19,...,234.51,170.0,169.0,0.01,NaN,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,NaN,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,NaN,NaN,0.69,2.43,7.57,0.74,2.72,NaN,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,NaN,14.24,...,149.64,79.0,77.0,0.01,NaN,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,NaN,3.0,19.58,1.0,0.0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         56
Jitter (local, absolute)               55
Jitter (rap)                           60
Jitter (ppq5)                          52
Jitter (ddp)                           59
Shimmer (local)                        37
Shimmer (local, dB)                    54
Shimmer (apq3)                         44
Shimmer (apq5)                         71
Shimmer (apq11)                        60
Shimmer (dda)                          51
AC                                     50
NTH                                    51
HTN                                    59
Median pitch                           54
Mean pitch                             69
Standard deviation                     52
Minimum pitch                          53
Maximum pitch                          52
Number of pulses                       57
Number of periods                      46
Mean period                            56
Standard deviation of period           53
Fraction of locally unvoiced frame

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 1456
persentase : 5.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         5.384615
Jitter (local, absolute)               5.288462
Jitter (rap)                           5.769231
Jitter (ppq5)                          5.000000
Jitter (ddp)                           5.673077
Shimmer (local)                        3.557692
Shimmer (local, dB)                    5.192308
Shimmer (apq3)                         4.230769
Shimmer (apq5)                         6.826923
Shimmer (apq11)                        5.769231
Shimmer (dda)                          4.903846
AC                                     4.807692
NTH                                    4.903846
HTN                                    5.673077
Median pitch                           5.192308
Mean pitch                             6.634615
Standard deviation                     5.000000
Minimum pitch                          5.096154
Maximum pitch                          5.000000
Number of pulses                       5.480769
Number of periods                      4

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,9)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
0,1.49,0.0,0.90,0.00,2.70,8.33,0.78,4.52,4.61,6.80,...,142.23,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,0.00,3.18,7.19,...,159.52,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,146.45,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0
3,2.50,0.0,1.16,1.63,3.47,13.51,0.00,5.26,8.77,16.78,...,182.71,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,182.82,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,96.15,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0
1036,1.76,0.0,0.00,0.69,2.43,7.57,0.74,2.72,0.00,11.93,...,121.49,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,0.00,14.24,...,95.92,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,90.05,451.37,75.0,70.0,0.01,0.0,0.00,3.0,19.58,1.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4642


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[1.21, 1.42, 3.09, 2.72, 1.17, 3.34, 2.27, 1.26, 1.36]
mean :  1.982222222222222
standar deviasi :  0.8285700906149873
fuzzy :  [(1.21, 0.6477430233739175), (1.42, 0.7943843270762234), (3.09, 0.40915596604353033), (2.72, 0.6727473949072088), (1.17, 0.6185268934940779), (3.34, 0.26118414798634365), (2.27, 0.94147372366431), (1.26, 0.6839663700633145), (1.36, 0.7543185467827352)]
fuzzy :  None
pembilang :  10.626807374237533
penyebut :  5.7835003933916616
rata-rata tertimbang :  1.8374352297753669
 
[2.08, 0.0, 2.75, 1.5, 1.54, 3.84, 2.95, 3.51, 0.73]
mean :  2.1
standar deviasi :  1.2074583038579658
fuzzy :  [(2.08, 0.9998628452369374), (0.0, 0.22041680205259345), (2.75, 0.8651264011928528), (1.5, 0.8838678197682575), (1.54, 0.8980435555687836), (3.84, 0.35409303133806225), (2.95, 0.7805528866136344), (3.51, 0.5057361369729472), (0.73, 0.5253930788202057)]
fuzzy :  None
pembilang :  12.988620170763665
penyebut :  6.033092557564274
rata-rata tertimbang :  2.1528958899326964
 
[

[0.0, 3.45, 2.5, 1.14, 3.05, 3.56, 2.11, 2.47, 2.23]
mean :  2.278888888888889
standar deviasi :  1.0653162748367728
fuzzy :  [(0.0, 0.10149158100105889), (3.45, 0.5465241347387412), (2.5, 0.9786930819406481), (1.14, 0.5647414458855119), (3.05, 0.769557219996741), (3.56, 0.48529164464676483), (2.11, 0.9875133953693838), (2.47, 0.9840393475623848), (2.23, 0.9989476515440414)]
fuzzy :  None
pembilang :  15.792717709594006
penyebut :  6.416799502685276
rata-rata tertimbang :  2.461151809868009
 
[0.0, 1.52, 1.36, 1.22, 3.49, 3.09, 3.66, 2.24, 1.54]
mean :  2.013333333333333
standar deviasi :  1.1391907459049848
fuzzy :  [(0.0, 0.20980491775339088), (1.52, 0.9105021608377555), (1.36, 0.8483705074751963), (1.22, 0.7846936541976086), (3.49, 0.4316957165245537), (3.09, 0.6398156379277194), (3.66, 0.35183807341934975), (2.24, 0.980401784567605), (1.54, 0.9173088843296706)]
fuzzy :  None
pembilang :  11.875204832642032
penyebut :  6.07443133703285
rata-rata tertimbang :  1.9549492246697546
 
[0

[0.0, 3.4, 1.42, 2.81, 1.52, 4.26, 1.5, 4.49, 3.28]
mean :  2.52
standar deviasi :  1.4120670427898718
fuzzy :  [(0.0, 0.2034641895836358), (3.4, 0.8235187954698758), (1.42, 0.7383118997934511), (2.81, 0.9791339884223811), (1.52, 0.7782288544446941), (4.26, 0.4680761895600201), (1.5, 0.7703854382982216), (4.49, 0.37791874492036265), (3.28, 0.8651742662623787)]
fuzzy :  None
pembilang :  15.466850651533152
penyebut :  6.004212366755022
rata-rata tertimbang :  2.5759999325094185
 
[0.0, 1.32, 1.53, 1.12, 2.05, 3.33, 1.7, 1.41, 2.5759999325094185]
mean :  1.670666659167713
standar deviasi :  0.883354205904156
fuzzy :  [(0.0, 0.16724806897304323), (1.32, 0.9242385100318891), (1.53, 0.9874024075103871), (1.12, 0.8234255884094072), (2.05, 0.9119292667845509), (3.33, 0.17134141865348082), (1.7, 0.9994488642765953), (1.41, 0.9574002314121504), (2.5759999325094185, 0.5914746447000752)]
fuzzy :  None
pembilang :  10.66561513716571
penyebut :  6.53390900075158
rata-rata tertimbang :  1.6323482827

[0.0, 1.08, 1.71, 3.31, 1.05, 1.45, 1.66, 1.12, 3.84]
mean :  1.691111111111111
standar deviasi :  1.118952331865685
fuzzy :  [(0.0, 0.3191968231383562), (1.08, 0.8614638792816123), (1.71, 0.999857543034529), (3.31, 0.35116750918737266), (1.05, 0.8486379660072312), (1.45, 0.9770540890327725), (1.66, 0.9996135894960957), (1.12, 0.8778850732696168), (3.84, 0.15820437772304793)]
fuzzy :  None
pembilang :  9.360394788110495
penyebut :  6.393080850170634
rata-rata tertimbang :  1.464144597492563
 
[0.0, 0.57, 0.25, 0.42, 0.6293007516833709, 0.84, 0.34, 1.76, 1.26]
mean :  0.6743667501870412
standar deviasi :  0.5135156399488588
fuzzy :  [(0.0, 0.42223055490722694), (0.57, 0.9795607317518739), (0.25, 0.7107539776003997), (0.42, 0.884555532829367), (0.6293007516833709, 0.996156917004882), (0.84, 0.9493164387483409), (0.34, 0.8089931560067353), (1.76, 0.1070436923382283), (1.26, 0.5219241013711884)]
fuzzy :  None
pembilang :  3.6529384797866387
penyebut :  6.380535102558243
rata-rata tertimban

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  5
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  5
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  5
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  5
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), 

...

[0.0, 3.584486084361217, 6.223917908675661, 13.62, 33.06, 8.19, 4.575021537912938, 10.851652403596356, 14.84435697902705]
mean :  10.549937212619248
standar deviasi :  9.15583198472532
fuzzy :  [(0.0, 0.5148974439609131), (3.584486084361217, 0.7487479241470695), (6.223917908675661, 0.8943922806876577), (13.62, 0.9453393365296998), (33.06, 0.04870767997553956), (8.19, 0.967330858291303), (4.575021537912938, 0.8082301249221383), (10.851652403596356, 0.999457242526295), (14.84435697902705, 0.8958464678907718)]
fuzzy :  None
pembilang :  58.50043562591654
penyebut :  6.822949358931388
rata-rata tertimbang :  8.574068566013642
 
[0.0, 0.03695151106584924, 0.01975682939285691, 0.10514302361592368, 0.012675021890585034, 0.016901379980979316, 3.608915859474233, 0.0, 53.19]
mean :  6.332260402824492
standar deviasi :  16.604336563791172
fuzzy :  [(0.0, 0.9298696275431633), (0.03695151106584924, 0.9306567452622028), (0.01975682939285691, 0.9302909652530834), (0.10514302361592368, 0.9320989487484

[0.0, 20.46, 0.69, 1.7154404262100529, 0.9019165630553021, 0.15780998550258102, 0.6087843002725999, 0.17877739807685988, 1.874082630812089]
mean :  2.954090144881054
standar deviasi :  6.220429794607767
fuzzy :  [(0.0, 0.8933704679166968), (20.46, 0.01907035042186236), (0.69, 0.935913172480255), (1.7154404262100529, 0.9803716547862025), (0.9019165630553021, 0.9470397467617048), (0.15780998550258102, 0.9039069254217581), (0.6087843002725999, 0.9313971732653028), (0.17877739807685988, 0.9052723255034573), (1.874082630812089, 0.985042147115882)]
fuzzy :  None
pembilang :  6.289437586080361
penyebut :  7.501383963673121
rata-rata tertimbang :  0.8384369626375824
 
[0.0, 25.79, 15.094133533609728, 24.89, 10.12, 14.255014385908465, 0.0, 14.04, 5.06]
mean :  12.138794213279802
standar deviasi :  8.911049181909581
fuzzy :  [(0.0, 0.3954530774810694), (25.79, 0.30934421151183594), (15.094133533609728, 0.9464948195066772), (24.89, 0.3592656621945296), (10.12, 0.9746667226803876), (14.25501438590

[0.0, 11.0, 1.0, 11.0, 1.0, 1.0, 20.0, 1.0, 1.0]
mean :  5.222222222222222
standar deviasi :  6.679617051195172
fuzzy :  [(0.0, 0.7366920202179997), (11.0, 0.687934306006101), (1.0, 0.8189289715410759), (11.0, 0.687934306006101), (1.0, 0.8189289715410759), (1.0, 0.8189289715410759), (20.0, 0.08654986320594188), (1.0, 0.8189289715410759), (1.0, 0.8189289715410759)]
fuzzy :  None
pembilang :  20.96019685395844
penyebut :  6.293755353141522
rata-rata tertimbang :  3.330316429204096
 
[46.0, 5.0, 20.0, 24.0, 1.0, 20.0, 1.0, 5.0, 5.0]
mean :  14.11111111111111
standar deviasi :  14.051053297366431
fuzzy :  [(46.0, 0.07614987132565243), (5.0, 0.8104154046723854), (20.0, 0.9159293404836949), (24.0, 0.7806491985788951), (1.0, 0.6470721130309892), (20.0, 0.9159293404836949), (1.0, 0.6470721130309892), (5.0, 0.8104154046723854), (5.0, 0.8104154046723854)]
fuzzy :  None
pembilang :  72.32602376236906
penyebut :  6.414048190951072
rata-rata tertimbang :  11.27618964017241
 
[40.0, 11.0, 1.0, 11.0,

[0.0, 1.0, 1.0, 1.0, 1.0, 55.0, 55.0, 1.0, 11.0]
mean :  14.0
standar deviasi :  22.140962540554153
fuzzy :  [(0.0, 0.8188219559908992), (1.0, 0.8416820618940224), (1.0, 0.8416820618940224), (1.0, 0.8416820618940224), (1.0, 0.8416820618940224), (55.0, 0.18008059310356075), (55.0, 0.18008059310356075), (1.0, 0.8416820618940224), (11.0, 0.9908634357333639)]
fuzzy :  None
pembilang :  34.916773343928796
penyebut :  6.3782568874014975
rata-rata tertimbang :  5.474344160219908
 
[0.0, 55.0, 1.0, 1.0, 1.0, 12.0, 55.0, 1.0, 12.0]
mean :  15.333333333333334
standar deviasi :  21.669230617526676
fuzzy :  [(0.0, 0.7785446093304643), (55.0, 0.1872542967503861), (1.0, 0.8035289159730159), (1.0, 0.8035289159730159), (1.0, 0.8035289159730159), (12.0, 0.9882394103459853), (55.0, 0.1872542967503861), (1.0, 0.8035289159730159), (12.0, 0.9882394103459853)]
fuzzy :  None
pembilang :  47.52983415473819
penyebut :  6.34364768741527
rata-rata tertimbang :  7.49250848987553
 
[0.0, 1.0, 23.0, 1.0, 1.0, 40.0,

[0.0, 5.0, 26.0, 7.356963184719059, 1.0, 46.0, 1.0, 1.0, 1.0]
mean :  9.817440353857673
standar deviasi :  14.935735265862407
fuzzy :  [(0.0, 0.8057294711515629), (5.0, 0.9493173730165507), (26.0, 0.5560468015311596), (7.356963184719059, 0.9865237967882771), (1.0, 0.8400922317108912), (46.0, 0.05318020392551788), (1.0, 0.8400922317108912), (1.0, 0.8400922317108912), (1.0, 0.8400922317108912)]
fuzzy :  None
pembilang :  32.26828126613091
penyebut :  6.711166573256632
rata-rata tertimbang :  4.808147870255073
 
[0.0, 46.0, 46.0, 1.0, 1.0, 46.0, 1.0, 23.0, 1.0]
mean :  18.333333333333332
standar deviasi :  20.698899595013366
fuzzy :  [(0.0, 0.6755654866065343), (46.0, 0.409347612495949), (46.0, 0.409347612495949), (1.0, 0.7042752264158701), (1.0, 0.7042752264158701), (46.0, 0.409347612495949), (1.0, 0.7042752264158701), (23.0, 0.9749078720732594), (1.0, 0.7042752264158701)]
fuzzy :  None
pembilang :  81.72995248778942
penyebut :  5.695617101831122
rata-rata tertimbang :  14.34962200347237

In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
0,1.49,0.0,0.900000,0.790919,2.70,8.33,0.780000,4.520000,4.610000,6.800000,...,142.23,187.580000,160.0,159.0,0.010000,0.0,13.687392,0.004864,5.878369,23.0
1,0.73,0.0,0.350000,0.380000,1.06,5.86,0.640000,4.900387,3.180000,7.190000,...,159.52,234.510000,170.0,169.0,0.010000,0.0,2.250000,1.039718,14.844357,23.0
2,1.22,0.0,0.730000,0.670000,2.20,8.72,0.880000,4.350000,5.170000,7.550000,...,146.45,211.440000,1431.0,1427.0,0.010000,0.0,10.660000,1.000000,0.180000,23.0
3,2.50,0.0,1.160000,1.630000,3.47,13.51,1.060778,5.260000,8.770000,16.780000,...,182.71,220.230000,94.0,92.0,0.000542,0.0,14.755172,1.000000,0.062945,23.0
4,3.51,0.0,1.720000,1.540000,5.15,9.11,1.040000,3.100000,4.930000,12.820000,...,182.82,225.160000,117.0,114.0,0.000024,0.0,18.180000,1.000000,13.320000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.720000,2.510000,8.16,9.85,0.970000,4.590000,6.810000,8.300000,...,96.15,158.590000,63.0,60.0,0.010000,0.0,24.190000,2.000000,15.110000,1.0
1036,1.76,0.0,1.101902,0.690000,2.43,7.57,0.740000,2.720000,6.146358,11.930000,...,121.49,143.800000,47.0,44.0,0.010000,0.0,46.050000,1.000000,25.790000,1.0
1037,2.91,0.0,1.140000,1.270000,3.42,15.49,1.460000,6.430000,8.105672,14.240000,...,95.92,149.640000,79.0,77.0,0.010000,0.0,20.000000,1.000000,4.570000,1.0
1038,3.79,0.0,1.790000,1.480000,5.36,15.70,1.370000,6.370000,11.090000,18.820000,...,90.05,451.370000,75.0,70.0,0.010000,0.0,21.487668,3.000000,19.580000,1.0


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000
mean,2.651658,0.0,1.240694,1.333734,3.702720,12.859926,1.186826,5.677199,7.879762,12.083487,...,134.559248,233.972597,109.376958,105.676219,7.838260e-03,0.00001,28.845298,1.269395,14.515909,12.819589
std,1.745469,0.0,0.968631,1.130553,2.897645,5.378013,0.412022,2.979340,4.740461,5.779113,...,46.479987,120.980935,149.853214,149.145244,3.952459e-03,0.00031,19.535305,1.478943,13.796049,15.638868
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,67.960000,85.540000,5.000000,4.000000,7.001861e-09,0.00000,0.160000,0.000006,0.002297,0.886295
25%,1.510000,0.0,0.637500,0.670000,1.850760,9.390000,0.950000,3.740000,5.212500,8.330000,...,101.918362,143.580000,42.016602,40.494875,9.458199e-03,0.00000,11.520065,0.337280,3.653356,1.000000
50%,2.350000,0.0,1.040000,1.110000,3.080000,12.260000,1.175000,5.115000,7.002047,11.320000,...,127.390000,195.590000,64.500000,62.000000,1.000000e-02,0.00000,27.270000,1.000000,9.874199,5.000000
75%,3.350000,0.0,1.590000,1.660000,4.762500,15.397500,1.398909,6.852500,9.222500,14.855000,...,158.107500,262.507500,112.250000,109.000000,1.000000e-02,0.00000,42.327500,1.165366,22.257500,23.000000
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,42.000000,...,452.080000,597.970000,1490.000000,1489.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000


# Klasifikasi

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [27]:
x = dummy.values
y = df.iloc[:, -1].values

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)

In [29]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [30]:
classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[133   8]
 [ 14 105]]
0.9153846153846154


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [31]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[132   9]
 [  9 110]]
0.9307692307692308


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [32]:
classifier = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[134   7]
 [ 13 106]]
0.9230769230769231


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [33]:
classifier = KNeighborsClassifier(n_neighbors=9, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[134   7]
 [ 14 105]]
0.9192307692307692


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
